In [52]:
from astropy.io import fits
import numpy as np
import pandas as pd

## Antenna Layout

* fitsrec from numpy.record
* 

Keys
* ANNAME
* STABXYZ
* ORBPARM
* NOSTA
* MNTSTA
* STAXOF
* DIAMETER
* BEAMFWHM (num_ant, 4)
* POLTYA
* POLAA
* POLCALA
* POLTYB
* POLAB
* POLCALB

In [ ]:
f = fits.open("../../test.uvf")

In [475]:
f[0].header

SIMPLE  =                    T /                                                
BITPIX  =                  -32 /                                                
NAXIS   =                    7 /                                                
NAXIS1  =                    0 /No standard image just group                    
NAXIS2  =                    3 /                                                
NAXIS3  =                    4 /                                                
NAXIS4  =                    1 /                                                
NAXIS5  =                    4 /                                                
NAXIS6  =                    1 /                                                
NAXIS7  =                    1 /                                                
EXTEND  =                    T /This is the antenna file                        
BLOCKED =                    T /Tape may be blocked                             
OBJECT  = '0316+413'        

In [481]:
f[0].data

GroupData([(       -0.00576364,        0.00252975,         0.00464756,  258., 2459184.91938657, 0., 65632., [[[[[[ 8.52234268e+00,  2.24202633e-01,  1.19211438e+03], [ 8.50443077e+00,  2.24619582e-01,  1.68925037e+03], [-1.94507726e-02,  2.78397091e-03,  9.93933301e+03], [-1.08099476e-01, -4.00930531e-02,  8.64082825e+02]]], [[[ 8.59763432e+00,  1.51146427e-01,  2.82057642e+03], [ 8.50234413e+00,  1.28182277e-01,  1.56099377e+03], [-9.57976580e-02,  1.41388290e-02,  2.11542773e+03], [-8.30844268e-02,  1.17492482e-01,  7.50006958e+02]]], [[[ 8.54740429e+00,  9.27989185e-02,  1.14010986e+03], [ 8.55224609e+00,  8.88005644e-02,  5.79439941e+02], [-1.17488757e-01, -4.08132896e-02,  3.58409814e+03], [-2.28986852e-02,  6.26851097e-02,  2.44200269e+03]]], [[[ 8.62037277e+00,  7.69680813e-02,  1.79620764e+03], [ 8.61219025e+00,  3.68038425e-03,  5.04876495e+02], [-7.21002445e-02, -3.58661935e-02,  1.45340515e+03], [ 1.21019691e-01,  6.10621423e-02,  1.59631335e+03]]]]]]),
           (       -0

In [478]:
f[0].data.columns

ColDefs(
    name = 'UU--'; format = 'E'; bscale = 6.57570277823e-11; bzero = 0.0
    name = 'VV--'; format = 'E'; bscale = 6.57570277823e-11; bzero = 0.0
    name = 'WW--'; format = 'E'; bscale = 6.57570277823e-11; bzero = 0.0
    name = 'BASELINE'; format = 'E'; bscale = 1.0; bzero = 0.0
    name = 'DATE'; format = 'E'; bscale = 1.0; bzero = 2459184.5
    name = '_DATE'; format = 'E'; bscale = 1.0; bzero = 0.0
    name = 'INTTIM'; format = 'E'; bscale = 1.0; bzero = 0.0
    name = 'DATA'; format = '48E'; bscale = 1.0; dim = (1, 1, 4, 1, 4, 3)
)

In [470]:
f[0].data

(10,)

In [473]:
def create_time_hdu(tba):
    TIME = np.ones(10, dtype=">f4") # placeholder, read from sim conf
    col1 = fits.Column(name='TIME', format='1E', unit="days", array=TIME)

    TIME_INTERVAL = np.ones(10, dtype=">f4") # placeholder, read from sim conf
    col2 = fits.Column(name='TIME INTERVAL', format='1E', unit="days", array=TIME_INTERVAL)

    SOURCE_ID = np.ones(10, dtype=">i4") # always the same source
    col3 = fits.Column(name='SOURCE ID', format='1J', unit=" ", array=SOURCE_ID)

    SUBARRAY = np.ones(10, dtype=">i4") # always same array
    col4 = fits.Column(name='SUBARRAY', format='1J', unit=" ", array=SUBARRAY)

    FREQ_ID = np.ones(10, dtype=">i4") # always same frequencies
    col5 = fits.Column(name='FREQ ID', format='1J', unit=" ", array=FREQ_ID)

    START_VIS = np.array([1, 733, 1489, 2245, 3001, 3901, 4801, 5746, 6646, 7546], dtype=">i4") # read from baselines
    col6 = fits.Column(name='START VIS', format='1J', unit=" ", array=START_VIS)

    END_VIS = np.array([ 732, 1488, 2244, 3000, 3900, 4800, 5745, 6645, 7545, 8418], dtype=">i4") # read from baselines)
    col7 = fits.Column(name='END VIS', format='1J', unit=" ", array=END_VIS)

    coldefs_time = fits.ColDefs([col1, col2, col3, col4, col5, col6, col7])
    hdu_time = fits.BinTableHDU.from_columns(coldefs_time)
    
    # add additional keywords 
    hdu_time.header["EXTNAME"] = ("AIPS NX", "AIPS NX")
    hdu_time.header["EXTVER"] = (1, "Version number of table")
    
    # add comments
    hdu_time.header.comments["TTYPE1"] = "Center time of interval"
    hdu_time.header.comments["TTYPE2"] = "Duration of interval"
    hdu_time.header.comments["TTYPE3"] = "Source number"
    hdu_time.header.comments["TTYPE4"] = "Subarray"
    hdu_time.header.comments["TTYPE5"] = "Frequency setup ID number"
    hdu_time.header.comments["TTYPE6"] = "First visibility number"
    hdu_time.header.comments["TTYPE7"] = "End visibility number"
    
    return hdu_time

In [474]:
time_hdu = create_time_hdu(None)
time_hdu.data

FITS_rec([(1., 1., 1, 1, 1,    1,  732), (1., 1., 1, 1, 1,  733, 1488),
          (1., 1., 1, 1, 1, 1489, 2244), (1., 1., 1, 1, 1, 2245, 3000),
          (1., 1., 1, 1, 1, 3001, 3900), (1., 1., 1, 1, 1, 3901, 4800),
          (1., 1., 1, 1, 1, 4801, 5745), (1., 1., 1, 1, 1, 5746, 6645),
          (1., 1., 1, 1, 1, 6646, 7545), (1., 1., 1, 1, 1, 7546, 8418)],
         dtype=(numpy.record, [('TIME', '<f4'), ('TIME INTERVAL', '<f4'), ('SOURCE ID', '<i4'), ('SUBARRAY', '<i4'), ('FREQ ID', '<i4'), ('START VIS', '<i4'), ('END VIS', '<i4')]))

In [438]:
def create_frequency_hdu(tba):
    FRQSEL = np.array([1], dtype=">i4")
    col1 = fits.Column(name='FRQSEL', format='1J', unit=" ", array=FRQSEL)

    IF_FREQ = np.array([[0.00e+00, 8.00e+07, 1.44e+08, 2.08e+08]], dtype=">f8") # placeholder, read from sim conf
    col2 = fits.Column(name='IF FREQ', format=str(IF_FREQ.shape[-1])+'D', unit="Hz", array=IF_FREQ)
    print(col2)

    CH_WIDTH = np.array([[64000000., 64000000., 64000000., 64000000.]], dtype=">f4") # placeholder, read from sim conf
    col3 = fits.Column(name='CH WIDTH', format=str(CH_WIDTH.shape[-1])+'E', unit="Hz", array=CH_WIDTH)

    TOTAL_BANDWIDTH = np.array([[64000000., 64000000., 64000000., 64000000.]], dtype=">f4") # placeholder, read from sim conf
    col4 = fits.Column(name='TOTAL BANDWIDTH', format=str(TOTAL_BANDWIDTH.shape[-1])+'E', unit="Hz", array=TOTAL_BANDWIDTH)

    SIDEBAND = np.zeros((1, IF_FREQ.shape[-1]))
    SIDEBAND[IF_FREQ>=0] = 1
    SIDEBAND[IF_FREQ<0] = -1
    col5 = fits.Column(name='SIDEBAND', format=str(SIDEBAND.shape[-1])+'J', unit=" ", array=SIDEBAND)

    RXCODE = np.chararray(1, itemsize=32, unicode=True)
    RXCODE[:] = ""
    col6 = fits.Column(name='RXCODE', format='32A', unit=" ", array=RXCODE)

    coldefs_freq = fits.ColDefs([col1, col2, col3, col4, col5, col6])
    hdu_freq = fits.BinTableHDU.from_columns(coldefs_freq)
    
    # add additional keywords 
    hdu_freq.header["EXTNAME"] = ("AIPS FQ", "AIPS FQ")
    hdu_freq.header["EXTVER"] = (1, "Version number of table")
    hdu_freq.header["NO_IF"] = (IF_FREQ.shape[-1], "Number IFs (n IF)")
    
    # add comments
    hdu_freq.header.comments["TTYPE1"] = "Frequency setup ID number"
    hdu_freq.header.comments["TTYPE2"] = "Frequency offset"
    hdu_freq.header.comments["TTYPE3"] = "Spectral channel separation"
    hdu_freq.header.comments["TTYPE4"] = "Total width of spectral window"
    hdu_freq.header.comments["TTYPE5"] = "Sideband"
    hdu_freq.header.comments["TTYPE6"] = "No one knows..."
    
    return hdu_freq

In [439]:
freq_hdu = create_frequency_hdu(None)
freq_hdu.header

name = 'IF FREQ'; format = '4D'; unit = 'Hz'


XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  116 / length of dimension 1                          
NAXIS2  =                    1 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                    6 / number of table fields                         
TTYPE1  = 'FRQSEL  '           / Frequency setup ID number                      
TFORM1  = '1J      '                                                            
TUNIT1  = '        '                                                            
TTYPE2  = 'IF FREQ '           / Frequency offset                               
TFORM2  = '4D      '        

In [310]:
def create_antenna_hdu(layout_txt):
    array = pd.read_csv(layout_txt, sep=" ")
    
    ANNAME = np.chararray(len(array), itemsize=8, unicode=True)
    ANNAME[:] = array["station_name"].values
    col1 = fits.Column(name='ANNAME', format='8A', array=ANNAME)
    
    STABXYZ = np.array([array["X"], array["Y"], array["Z"]], dtype=">f8").T
    col2 = fits.Column(name='STABXYZ', format='3D', unit="METERS", array=STABXYZ)

    ORBPARM = np.array([], dtype=">f8")
    col3 = fits.Column(name='ORBPARM', format='0D', unit=" ", array=ORBPARM)

    NOSTA = np.arange(len(array), dtype=">i4")
    col4 = fits.Column(name='NOSTA', format='1J', unit=" ", array=NOSTA)

    MNTSTA = np.zeros(len(array), dtype=">i4")
    col5 = fits.Column(name='MNTSTA', format='1J', unit=" ", array=MNTSTA)

    STAXOF = np.zeros(len(array), dtype=">f4")
    col6 = fits.Column(name='STAXOF', format='1E', unit="METERS", array=STAXOF)

    DIAMETER = np.array(array["dish_dia"].values, dtype=">f4")
    col7 = fits.Column(name='DIAMETER', format='1E', unit="METERS", array=DIAMETER)

    BEAMFWHM = np.zeros((len(array), 4), dtype=">f4")
    col8 = fits.Column(name='BEAMFWHM', format='4E', unit="DEGR/M", array=BEAMFWHM)

    POLTYA = np.chararray(len(array), itemsize=1, unicode=True)
    POLTYA[:] = "R"
    col9 = fits.Column(name='POLTYA', format='1A', unit=" ", array=POLTYA)

    POLAA = np.zeros(len(array), dtype=">f4")
    col10 = fits.Column(name='POLAA', format='1E', unit="DEGREES", array=POLAA)

    POLCALA = np.zeros((len(array), 8), dtype=">f4")
    col11 = fits.Column(name='POLCALA', format='8E', unit=" ", array=POLCALA)

    POLTYB = np.chararray(len(array), itemsize=1, unicode=True)
    POLTYB[:] = "L"
    col12 = fits.Column(name='POLTYB', format='1A', unit=" ", array=POLTYB)

    POLAB = np.zeros(len(array), dtype=">f4")
    col13 = fits.Column(name='POLAB', format='1E', unit="DEGREES", array=POLAB)

    POLCALB = np.zeros((len(array), 8), dtype=">f4")
    col14 = fits.Column(name='POLCALB', format='8E', unit=" ", array=POLCALB)

    coldefs_ant = fits.ColDefs([col1, col2, col3, col4, col5, col6, col7, col8, col9, col10, col11, col12, col13, col14])
    hdu_ant = fits.BinTableHDU.from_columns(coldefs_ant)
    
    # add additional keywords
    hdu_ant.header["EXTNAME"] = ("AIPS AN", "AIPS table file")
    hdu_ant.header["EXTVER"] = (1, "Version number of table")
    hdu_ant.header["ARRAYX"] = (0, "x coordinate of array center (meters)")
    hdu_ant.header["ARRAYY"] = (0, "y coordinate of array center (meters)")
    hdu_ant.header["ARRAYZ"] = (0, "z coordinate of array center (meters)")
    hdu_ant.header["GSTIA0"] = (119.85, "GST at 0h on reference date (degrees)") # how to calculate??
    hdu_ant.header["DEGPDY"] = (360.98564497329994, "Earth's rotation rate (degrees/day)")
    hdu_ant.header["FREQ"] = (227297000000.0, "Reference frequency (Hz)") # read from simulations
    hdu_ant.header["RDATE"] = ("2015-04-08T00:00:00.0", "Reference date") # where do we find this?
    hdu_ant.header["POLARX"] = (0, "x coordinate of North Pole (arc seconds)") # missing, 0.10819000005722046 MOJAVE
    hdu_ant.header["POLARY"] = (0, "y coordinate of North Pole (arc seconds)") # missing 0.28815001249313354 MOJAVE
    hdu_ant.header["UT1UTC"] = (0, "UT1 - UTC (sec)") # missing
    hdu_ant.header["DATUTC"] = (0, "time system - UTC (sec)") # missing
    hdu_ant.header["TIMSYS"] = ("UTC", "Time system")
    hdu_ant.header["ARRNAM"] = ("EHT", "Array name") # read from layout file
    hdu_ant.header["XYZHAND"] = ("RIGHT", "Handedness of station coordinates")
    hdu_ant.header["FRAME"] = ("????", "Coordinate frame")
    hdu_ant.header["NUMORB"] = (0, "Number orbital parameters in table (n orb)")
    hdu_ant.header["NOPCAL"] = (2, "Number of polarization calibration values / IF(n pcal)")
    hdu_ant.header["NO_IF"] = (4, "Number IFs (n IF)")
    hdu_ant.header["FREQID"] = (-1, "Frequency setup number")
    hdu_ant.header["IATUTC"] = (33, "No one knows.....") # how to calculate??
    hdu_ant.header["POLTYPE"] = (" ", "Type of polarization calibration")

    # add comments
    hdu_ant.header.comments["TTYPE1"] = "Antenna name"
    hdu_ant.header.comments["TTYPE2"] = "Antenna station coordinates (x, y, z)"
    hdu_ant.header.comments["TTYPE3"] = "Orbital parameters"
    hdu_ant.header.comments["TTYPE4"] = "Antenna number"
    hdu_ant.header.comments["TTYPE5"] = "Mount type"
    hdu_ant.header.comments["TTYPE6"] = "Axis offset"
    hdu_ant.header.comments["TTYPE7"] = "Antenna diameter"
    hdu_ant.header.comments["TTYPE8"] = "Antenna beam FWHM"
    hdu_ant.header.comments["TTYPE9"] = "R, L, feed A"
    hdu_ant.header.comments["TTYPE10"] = "Position angle feed A"
    hdu_ant.header.comments["TTYPE11"] = "Calibration parameters feed A"
    hdu_ant.header.comments["TTYPE12"] = "R, L, polarization 2"
    hdu_ant.header.comments["TTYPE13"] = "Position angle feed B"
    hdu_ant.header.comments["TTYPE14"] = "Calibration parameters feed B"
    
    return hdu_ant

In [480]:
path = "../vipy/layouts/eht.txt"
ant_hdu = create_antenna_hdu(path)
ant_hdu.header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  138 / length of dimension 1                          
NAXIS2  =                    8 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                   14 / number of table fields                         
TTYPE1  = 'ANNAME  '           / Antenna name                                   
TFORM1  = '8A      '                                                            
TTYPE2  = 'STABXYZ '           / Antenna station coordinates (x, y, z)          
TFORM2  = '3D      '                                                            
TUNIT2  = 'METERS  '        

In [322]:
ant_hdu.data["POLAB"]

array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [296]:
cel = fits.open("../../celestial-01-05.uvfits")

In [476]:
cel[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    7 / number of data axes                            
NAXIS1  =                    0 / length of data axis 1                          
NAXIS2  =                    3 / length of data axis 2                          
NAXIS3  =                    4 / length of data axis 3                          
NAXIS4  =                    1 / length of data axis 4                          
NAXIS5  =                    1 / length of data axis 5                          
NAXIS6  =                    1 / length of data axis 6                          
NAXIS7  =                    1 / length of data axis 7                          
EXTEND  =                    T / FITS dataset may contain extensions            
GROUPS  =                    T / random group records are present               
PCOUNT  =                   

In [324]:
f[3].header

XTENSION= 'BINTABLE'           / Extension type                                 
BITPIX  =                    8 / Binary data                                    
NAXIS   =                    2 / Table is a matrix                              
NAXIS1  =                  138 / Width of table in bytes                        
NAXIS2  =                   10 / Number of entries in table                     
PCOUNT  =                    0 / Random parameter count                         
GCOUNT  =                    1 / Group count                                    
TFIELDS =                   14 / Number of fields in each row                   
EXTNAME = 'AIPS AN '           / AIPS table file                                
EXTVER  =                    1 / Version number of table                        
TFORM1  = '8A      '           / FORTRAN format of field  1                     
TTYPE1  = 'ANNAME          '   / Type (heading) of field  1                     
TUNIT1  = '        '        